In [ ]:
import numpy as np
%load_ext autoreload
%autoreload 0

Notebook by **Maxime Dion** <maxime.dion@usherbrooke.ca><br>
For the QSciTech-QuantumBC virtual workshop on gate-based quantum computing

## Tutorial for the VQE Activity

For this activity, make sure you can easily import your versions of `hamiltonian.py`, `pauli_string.py` and `mapping.py` that you have completed in the mapping tutorial. You will also need your verions of `evaluator.py` and `solver.py`. Placing this notebook in the same `path` as these files is the easiest way to acheive this. At the end of this notebook, you should be in good position to complete these 2 additionnal files.

The solution we suggest here is NOT mandatory. If you find ways to make it better and more efficient, go on and impress us! On the other hand, by completing all sections of this notebook you'll be able to :
- Prepare a Quantum State based on a varitional form (circuit);
- Measure qubits in the X, Y and Z basis;
- Estimate expectation value of Pauli String on a quantum state;
- Evaluate the expectation value of an Hamiltonian in the form of a Linear Combinaison of Pauli Strings;
- Run a minimization algorithm on the energy expectation fonction to find the ground state of a Hamiltonian;
- Dance to express your overwhelming sense of accomplishment

**Important**

When you modify and save a `*.py` file you need to re-import it so that your modifications can be taken into account when you re-execute a call. By adding the magic command `%autoreload` at the beginning of a cell, you make sure that the modifications you did to the `*.py` files are taken into account when you re-run a cell and that you can see the effect.

If you encounter unusual results, restart the kernel and try again.

**Note on numbering**

When you ask a question you can refer to the section name or the section number.

To enable the section numbering, please make sure you install [nbextensions](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html). It is available in the conda distribution. After installation it you need to enable the option 'Table of contents (2)'.

# Variationnal Quantum States

Every quantum circuit starts with all qubits in the state $|0\rangle$. In order to prepare a quantum state $|\psi\rangle$ we need to prepare a `QuantumCircuit` that will modify the states of the qubits in order to get this specific state. The action of a circuit can always be represented as a unitiary operator.

\begin{align}
    |\psi\rangle &= \hat{U} |0 \ldots 0\rangle
\end{align}

For a parametric state the `QuantumCircuit` and therefore the unitary $U$ will depend on some parameters that we wirte as $\boldsymbol{\theta}$.

\begin{align}
    |\psi(\boldsymbol{\theta})\rangle &= \hat{U}(\boldsymbol{\theta}) |0 \ldots 0\rangle
\end{align}

We will see 2 ways to define Parametrized Quantum Circuits that represent Variationnal Quantum States. For the first method we only need the `QuantumCircuit` class from `qiskit.circuit`.

In [ ]:
from qiskit.circuit import QuantumCircuit

## Generating function
The easiest way to generate a parametrized `QuantumCircuit` is to implement a function that takes parameters as arguments and returns a `QuantumCircuit`. Here is such a function that generates a 2 qubits QuantumCircuit.

In [ ]:
def example_2qubits_2params_quantum_circuit(theta,phi):
    qc = QuantumCircuit(2)
    qc.ry(theta,0)
    qc.rz(phi,0)
    qc.cx(0,1)
    return qc

To visualize this circuit we first need to call the generating function with dummy argument values for it to return a circuit. We can draw the circuit. The `'mpl'` option draws the circuit in a fancy way using `matplotlib`. If you are experiencing problems, you can remove this option.

In [ ]:
varform_qc = example_2qubits_2params_quantum_circuit
qc = varform_qc(1,2)
qc.draw('mpl')

## Using qiskit parameter

The other way to generate a parametrized `QuantumCircuit` is to use the `Parameter` class in `qiskit`.

In [ ]:
from qiskit.circuit import Parameter

Here is the same circuit as before done with this method.

In [ ]:
a = Parameter('a')
b = Parameter('b')
varform_qc = QuantumCircuit(2)
varform_qc.ry(a,0)
varform_qc.rz(b,0)
varform_qc.cx(0,1)

Done this way the parametrized circuit can be drawn right away.

In [ ]:
varform_qc.draw('mpl')

To see what are the parameters of a parametrized `QuantumCircuit` you can use

In [ ]:
varform_qc.parameters

To assign values to the different parameters we need to use the `QuantumCircuit.assign_paremeters()` method. This methods takes a `dict` as an argument containing the `Parameter`s and their `value`s.

In [ ]:
param_dict = {a : 1, b : 2}
qc = varform_qc.assign_parameters(param_dict)
qc.draw('mpl')

If you want to provide the parameter values as a `list` or a `np.array` you can build the `dict` directly. Just make sure that the order you use in `param_values` corresponds to the other of `varform_qc.parameters`.

In [ ]:
param_values = [1, 2]
param_dict = dict(zip(varform_qc.parameters,param_values))
print(param_dict)

## Varforms circuits for H2
Using the method of you choice, prepare 2 different 4-qubit `QuantumCircuit`s. 
- The first should take 1 parameter to cover the real coefficients state sub space spanned by $|0101\rangle$ and $|1010\rangle$.
- The second should take 3 parameters to cover the real coefficients state sub space spanned by $|0101\rangle$, $|0110\rangle$, $|1001\rangle$ and $|1010\rangle$.

Revisit the presentation to find such circuits.

In [ ]:
varform_4qubits_1param = QuantumCircuit(4)
a = Parameter('a')
"""
Your code here
"""

varform_4qubits_1param.draw('mpl')

In [ ]:
varform_4qubits_3params = QuantumCircuit(4)
a = Parameter('a')
b = Parameter('b')
c = Parameter('c')
"""
Your code here
"""

varform_4qubits_3params.draw('mpl')

# Estimation
The `Estimator` is an object that will help us to estimate the expectation value of an observable such as the Hamiltonian (in the for of a `LinearCombinaisonPauliString`) on a specific variational form and backend. To initialize an `Estimator` you should provide :

**Mandatory**
- A **variationnal form** that can create a `QuantumCircuit` given a set of `params`;
- A **backend** `qiskit.Backend` (a simulator or an actual device handle) on which to run the `QuantumCircuit`.

**Optional**
- `execute_opt` is a `dict` containing the optional arguments to pass to the `qiskit.execute` method (ex : `{'shots' : 1024}`);
- `record` can be a function used to record results of the estimation.

Once you'll have completed the code, the creation/usage of an `Estimator` such as `BasicEstimator` will go like this :

<code>
estimator = BasicEstimator(varform_qc, backend)<br>estimator.set_observable(observable)<br>expected_value = estimator.eval(params)
</code>

First you'll initialize the estimator.

Next, you'll provide the observables (`LinearCombinaisonPauliString`) you want to estimate the expectation value using the `set_observable()` method. 

Finally, you'll call the `eval()` method that will return the estimation of the observable's expected value. Mathematicaly, the use of this method corresponds to 

\begin{align}
E(\boldsymbol{\theta}).
\end{align}

We will now go through the different pieces neccessary to complete the `Estimator` class.

## State circuit

The first method we need to implement is `prepare_state_circuit()`. This method should take a `list` or a `ndarray` of parameters and replace then into the `varform` quantum circuit and return the `QuantumCircuit` that prepares the quantum state.

You can test you code here with the abstract class.

In [ ]:
from qiskit import Aer, execute
qasm_simulator = Aer.get_backend('qasm_simulator')
from estimator import Estimator

In [ ]:
%autoreload
varform = varform_4qubits_1param
backend = qasm_simulator
estimator = Estimator(varform,backend)
params = np.random.random(1)
state_circuit = estimator.prepare_state_circuit(params)

state_circuit.draw('mpl')

## Pauli Based Measurements
We want to estimate the expectation value of an observable for the quantum state we are preparing.

We have seen that even if a quantum computer can only measure qubits in the Z-basis, the X and Y-basis are accessible if we *rotate* the quantum state before measuring. 

Implement the `@staticmethod` : `diagonalizing_pauli_string_circuit()` in the `Estimator` class in file `Estimator.py` that returns a `QuantumCircuit` that performs a transformation which diagonalize the provided `PauliString`.

First we import the abstract class `Estimator` and the `PauliString` class.

In [ ]:
from pauli_string import PauliString

Test your code with the next cell.

In [ ]:
%autoreload
pauli_string = PauliString.from_str('ZIXY')
diagonalizing_circuit, diagonal_pauli_string = Estimator.diagonalizing_pauli_string_circuit(pauli_string)
print(diagonal_pauli_string) #should be 'ZIZZ'
diagonalizing_circuit.draw('mpl')

## Circuit and diagonal observables preperation (`BasicEstimator`)

The `diagonal_observables_and_circuits()` method is specific to the sub-types of `Estimator`. The two different types of `Estimator`s considered in this workshop are :
- The `BasicEstimator` will run a single `QuantumCircuit` for each `PauliString` present in the provided observable (`LinearCombinaisonPauliString`).
- The `BitwiseCommutingCliqueEstimator` will exploit Bitwise Commuting Clique to combine the estimation of Commuting `PauliString`s and reduce the number of different `QuantumCircuit` ran to estimate the expectation value of the observable. 

Implement the `diagonal_observables_and_circuits()` method in the `BasicEstimator` class in file `Estimator.py`. This method needs to convert each `PauliString` in the observable into a diagonal `PauliString` (of only `I` and `Z`) and a quantum circuit. The diagonal `PauliString` should then be combined with its associated coefficient into a  `LinearCombinaisonPauliString` of only one `PauliString` to form a diagonal observable. The quantum circuit should represent the transformation which convert the original `PauliString` into the respective diagonal `PauliString`. The method should return all the diagonal observables and respective quantum circuits into two `list`.

**Note** The reason the diagonal `PauliString` are returned as `LinearCombinaisonPauliString` of only one `PauliString` and not a single `LinearCombinaisonPauliString` of many `PauliString` is to facilitate the implementation of the similar methods for the `BitwiseCommutingCliqueEstimator`. You can try to implemente this second `Estimator` if you wish.

You can test your method on `2*ZIXY + 1*IXYZ`.

In [ ]:
from estimator import BasicEstimator

In [ ]:
%autoreload
observable = 2*PauliString.from_str('ZXZX') + 1*PauliString.from_str('IIZZ')
diagonal_observables, diagonalizing_circuits = BasicEstimator.diagonal_observables_and_circuits(observable)

for diagonal_observable in diagonal_observables:
    print(diagonal_observable)

You should get :

<code>
1 pauli strings for 4 qubits (Real, Imaginary)<br>
ZZZZ (+2.00000,+0.00000)<br>

1 pauli strings for 4 qubits (Real, Imaginary)<br>
IIZZ (+1.00000,+0.00000)<br>
</code>

## Set the observable

You should now be able to instantiate the `BasicEstimator` and to set an observable using `set_observable()`. This method is already implemented. It calls the `diagonal_observables_and_circuits()` method you just made and stores the `diagonal_observables` and the `diagonalizing_circuits` into the instance of the `Estimator`.

## Assemble circuits

Now that we have the state circuit and diagonalizing circuits we can assemble all the quantum circuits that will be executed in the estimation of the expectation value of the observable. The method `assemble_circuits()` assembles the state circuit with each one of the diagonalizing circuits to create as many circuits. This method should also apply the final measurements.

One you have implemented this method you can test it here:

In [ ]:
%autoreload

varform = varform_4qubits_1param
backend = qasm_simulator
estimator = BasicEstimator(varform, backend)

observable = 2*PauliString.from_str('ZXZX') + 1*PauliString.from_str('IIZZ')
estimator.set_observable(observable)
params = [0,]
state_circuit = estimator.prepare_state_circuit(params)
circuits = estimator.assemble_circuits(state_circuit)

You should then be able to execute these circuits...

In [ ]:
execute_opts = {'shots' : 1024, 'seed_simulator' : 1}
job = execute(circuits, backend=qasm_simulator, **execute_opts)
result = job.result()

... and obtain the results as counts.

In [ ]:
i = 0
counts = result.get_counts(circuits[i])
print(counts)

You should get :

<code>
{'0100': 245, '0000': 234, '0101': 272, '0001': 273}
</code>

## Diagonal pauli string eigenvalue

With these counts in hands, we can tackle the last steps of the estimation of the expectation value of an observable. First we need to compute the eigenvalue of a diagonal pauli string for a given basis state. This eigenvalue can only be `+1` or `-1`. This computation should be done in the `diagonal_pauli_string_eigenvalue()` method that you can implement now.

You can test your implementation with the following code which should run `-1`.

In [ ]:
%autoreload

diagonal_pauli_string = PauliString.from_str('ZIZZ')
state = '0110'
eigenvalue = Estimator.diagonal_pauli_string_eigenvalue(diagonal_pauli_string, state)
eigenvalue

## Diagonal pauli string expectation value

Let's now estimate the expectation value of a single diagonal `PauliString`. This needs to be done in the method `estimate_diagonal_pauli_string_expectation_value()`. Implement this method using the one you just implemented and the following equation
\begin{align}
    \langle \hat{\mathcal{P}} \rangle = \frac{1}{N_\text{tot}}\sum_{q} N_q \Lambda_q^{(\hat{\mathcal{P}})}
\end{align}
where $\Lambda_q^{(\hat{\mathcal{P}})}$ is the eigenvalue of the `PauliString` for a state $q$ and $N_q$ is the counts, i.e. the number of times this state was measured. Testing your implementation, you should get an expectation value of `0`.

In [ ]:
%autoreload

diagonal_pauli_string = PauliString.from_str('ZIZZ')
counts = {'0110' : 50, '1001' : 50}
pauli_string_expectation_value = Estimator.estimate_diagonal_pauli_string_expectation_value(diagonal_pauli_string, counts)
pauli_string_expectation_value

## Diagonal observable expectation value

This last step may seem a bit redondant, but it makes more sense if you try to implement the `BitwiseCommutingCliqueEstimator`. Anyhow, you need to implement the method `estimate_diagonal_observable_expectation_value()` so that it computes the expectation value of a diagonal observable by computing the linear combinaison of the diagonal pauli string expectation value, weighted by their respective coefficients.
\begin{align}
    \langle \hat{\mathcal{D}} \rangle  = \sum_i h_i \langle \hat{\mathcal{P}}_i \rangle 
\end{align}

In [ ]:
%autoreload

diagonal_observable = 2*PauliString.from_str('ZZZZ') + 1*PauliString.from_str('IIZZ')
counts = {'0110' : 50, '1001' : 50}
observable_expectation_value = Estimator.estimate_diagonal_observable_expectation_value(diagonal_observable, counts)

Again, you should get an expectation value of `0`.

## The `eval(params)` method

You have now all the pieces to complete the `eval(params)` method. This method should use many methods you've just implemented. Mathematicaly, it should return the value of the expression

\begin{align}
    E(\boldsymbol{\theta}) = \sum_i h_i \langle\psi(\boldsymbol{\theta}) | \hat{\mathcal{P}}_i | \psi(\boldsymbol{\theta}) \rangle.
\end{align}

In [ ]:
%autoreload

varform = varform_4qubits_1param
backend = qasm_simulator
execute_opts = {'shots' : 1024, 'seed_simulator' : 1}
evaluator = BasicEstimator(varform,backend,execute_opts = execute_opts)

observable = 2*PauliString.from_str('ZXZX') + 1*PauliString.from_str('IIZZ')
evaluator.set_observable(observable)
params = [0,]
expected_value = evaluator.eval(params)
print(expected_value)

Yes that's right, your code now returns an estimate of the expression

\begin{align}
    E(\theta) = \langle \psi(\theta) | \hat{\mathcal{H}} | \psi(\theta) \rangle.
\end{align} 

for

\begin{align}
    \hat{\mathcal{H}} = 2\times \hat{Z}\hat{X}\hat{Z}\hat{X} + 1\times \hat{I}\hat{I}\hat{Z}\hat{Z}
\end{align} 

and the varform `varform_4qubits_1param` with $\theta = 0$. The `evaluator.eval()` is now a method you can call like a function and it will return the energy $E(\theta)$.

Now comes the time to test this on the $\text{H}_2$ molecule Hamiltonian!

## The Hamiltonian evaluation test

We will now import the classes from the previous activity.

In [ ]:
from hamiltonian import MolecularFermionicHamiltonian
from mapping import JordanWigner

For ease of use we will import the integral values instead of using `pyscf`. We also import the Coulomb repulsion energy for later use. By now we are experts in building the Hamiltonian.

In [ ]:
with open('Integrals_sto-3g_H2_d_0.7350_no_spin.npz','rb') as f:
    out = np.load(f)
    h1_load_no_spin = out['h1']
    h2_load_no_spin = out['h2']
    energy_nuc =  out['energy_nuc']
    
molecular_hamiltonian = MolecularFermionicHamiltonian.from_integrals(h1_load_no_spin,h2_load_no_spin).include_spin()

We use the Jordan-Wigner mapping to the get the Hamiltonian for the H2 molecule with `d=0.735`. 

In [ ]:
%autoreload
mapping = JordanWigner()
qubit_hamiltonian = mapping.fermionic_hamiltonian_to_qubit_hamiltonian(molecular_hamiltonian).sort()
print(qubit_hamiltonian)

We build an evaluator, feed it with the Hamiltonian (`LinearCombinaisonPauliString`) of H2 and then we evaluate the energy. Use `params` in order that your `varform` prepares the state $|0101\rangle$.

In [ ]:
%autoreload
varform = varform_4qubits_1param
backend = qasm_simulator
execute_opts = {'shots' : 2048}
estimator = BasicEstimator(varform, backend,execute_opts = execute_opts)
estimator.set_observable(qubit_hamiltonian)
params = [0,]
expected_value = estimator.eval(params)
print(expected_value)

If your `varform` prepares the state $|0101\rangle$, you should get something around `-1.83`. This energy is already close to the ground state energy because the ground state is close to $|0101\rangle$, but still it's not the ground state. We need to find the `params` that will minimise the energy.

\begin{align}
    E_0 = \min_{\boldsymbol{\theta}} E(\boldsymbol{\theta})
\end{align}

# Solver

In a final step we need to implement a solver that will try to find the minimal energy. We will implement 2 solvers (the second is optionnal):
  1. Use the VQE algo in conjunction with a minimizer to try to minimize `estimator.eval()`.
  2. (optional) Use the `to_matrix()` method you implemented in the previous activity to find the exact value/solution.

## VQE Solver

Like any minimzation process this solver will need a couple of ingredients :
- A function to minimize, we will provide this with the evaluator
- A minimizer, an algorithm that generaly takes in a function and a set of starting parameters and returns the best guess for the optimal parameters that correspond to the minimal value of the function to minimize.
- A set of starting parameters.

### Minimizer

A minimizer that works OK for the VQE algorithm is the  Sequential Least SQuares Programming (SLSQP) algorithm. It's available in the `minimize` sub-module of [scipy](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html).

In [ ]:
from scipy.optimize import minimize

We will make a Lambda function with the minimizer so we can set all sorts of parameters before feeding it to the solver.

In [ ]:
minimizer = lambda fct, start_param_values : minimize(
    fct,
    start_param_values,
    method = 'SLSQP', 
    options = {'maxiter' : 5,'eps' : 1e-1, 'ftol' : 1e-4, 'disp' : True, 'iprint' : 2})

The `minimizer` now takes only 2 arguments : the function and the starting parameter's values. We also specify some options :
- A small value for the maximum number of iterations. You will find that running the VQE algorithm is expansive because of the `estimator.eval()` method. Either it's long to simulate on `qasm_simulator` or because it's running on an actual quantum computer.
- A `eps` of `0.1`. This is the size of the step the algorithm is going to change the values of the parameters to try to estimate the slope of the function. By the way, a lot of minimizing algorithms use the slope of the function to know in which direction is the minimum. Since our parameters are all angles in radians a value of 0.1 seems reasonnable. Play with this value if you like.
- A `ftol` value of `1e-4`. This is the goal for the precision of the value of the minimum value. The chemical accuracy is around 1 milli-Hartree.
- We set `iprint` to `2` so to see what is going on. For your final implementation you can set this to `0`.

Before implementing the `VQESolver` let's try this minimizer! The function is `estimator.eval` and we start with a parameter of `0`.

In [ ]:
minimization_result = minimizer(estimator.eval,[0,])

In the end you should get a minimal energy around `1.86` Hartree. Which is a bit smaller than what we had before minimizing. You can explore the `minimization_result` to retreive this value but also the set of optimal parameters.

In [ ]:
opt_params = minimization_result.x
opt_value = minimization_result.fun
print(opt_params)
print(opt_value)

### VQE Solver

Now you should be in good position to implement the `lowest_eig_value(lcps)` of the `VQESolver` class inside the `Solve.py` file. Test your method here.

In [ ]:
from solver import VQESolver

In [ ]:
%autoreload
vqe_solver = VQESolver(estimator,minimizer,[0,],name = 'vqe_solver')
opt_value = vqe_solver.lowest_eig_value(qubit_hamiltonian)

There is only one thing missing to have the complete molecular energy : the Coulomb repulsion energy from the nucleus. This value was loaded when we imported the integrals. Let's add it to the electronic energy.

In [ ]:
print('Ground state position estimate (vqe) : ', vqe_solver.last_opt_params)
print('Ground state energy estimate (electronic, vqe) : ', opt_value)
print('Ground state energy estimate (molecular, vqe) : ', opt_value + energy_nuc)

### The Eigenstate

What is the eigenstate? We can partially find out by using the `varform` with the parameters we have found and measure everything in the Z basis.

In [ ]:
eigenstate_qc = varform.copy()
eigenstate_qc.measure_all()

param_dict = dict(zip(eigenstate_qc.parameters,opt_params))
eigenstate_qc = eigenstate_qc.assign_parameters(param_dict)

eigenstate_qc.draw('mpl')

We now execute this circuit.

In [ ]:
execute_opts = {'shots' : 1024}
job = execute(eigenstate_qc,backend=qasm_simulator,**execute_opts)
result = job.result()
counts = result.get_counts(eigenstate_qc)

We will use the `plot_histogram` method from `qiskit.visualization` that takes the counts `dict` as an input. 

In [ ]:
from qiskit.visualization import plot_histogram

In [ ]:
plot_histogram(counts)

We see that the found solution is mostly the state $|0101\rangle$ which is the Hartree-Fock solution when the 2-body Hamiltonian is not present. Adding this 2-body physics shifts the energy down a bit by introducing a small contribution of $|1010\rangle$. The actual statevector has a `-` sign between these two states.

\begin{align}
\alpha_{0101}|0101\rangle - \alpha_{1010}|0101\rangle
\end{align}

But this is not something we can know from this. Fortunatly, H2 is a small system which can be solved exactly and we can find out this phase.

## Exact Solver (optional)

If you want to compare the value you get with the VQE algorithm it would be nice to have the exact value. If you were able to implement the `to_matrix()` method for `PauliString` and `LinearCombinaisonPauliString` then you can find the exact value of the ground state. All you need is to diagonalise the matrix reprensenting the whole Hamiltonian and find the lowest eigenvalue! Obviously this will not be possible to do for very large systems.

In [ ]:
hamiltonian_matrix = qubit_hamiltonian.to_matrix()
eig_values, eig_vectors = np.linalg.eigh(hamiltonian_matrix)
eig_order = np.argsort(eig_values)
eig_values = eig_values[eig_order]
eig_vectors = eig_vectors[:,eig_order]
ground_state_value, ground_state_vector = eig_values[0], eig_vectors[:,0]
print('Ground state vector (exact) : ', ground_state_vector)
print('Ground state energy (electronic, exact) : ', ground_state_value)
print('Ground state energy (molecular, exact) : ', ground_state_value + energy_nuc)

Now you can complete the `ExactSolver` in the `Solver.py` file.

In [ ]:
from solver import ExactSolver

In [ ]:
%autoreload
exact_solver = ExactSolver()
ground_state_value, ground_state_vector = exact_solver.lowest_eig_value(qubit_hamiltonian)
print('Ground state vector (exact) : ', ground_state_vector)
print('Ground state energy (electronic, exact) : ', ground_state_value)
print('Ground state energy (molecular, exact) : ', ground_state_value + energy_nuc)

What are the two basis states involved in the ground state? Let's plot the state vector using `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig,ax = plt.subplots(1,1)
i_max = np.argmax(np.abs(ground_state_vector))
state = ground_state_vector * np.sign(ground_state_vector[i_max])
ax.bar(range(len(state)),np.abs(state),color=(np.real(state) > 0).choose(['r','b']))
plt.xticks(range(len(state)),[f"{i:04b}" for i in range(len(state))], size='small',rotation=60);

# What's next?

Now that you can find the ground state for a specific H2 molecule configuration (`d = 0.735`), you should be able to do that for many configurations, say `d = 0.2` to `2.5`. Doing that will enable you to plot the so-called dissociation curve : energy vs distance. Do not forget to include the Coulomb repulsion energy of the nucleus!

You could also run your algorithm on a noisy backend, either a noisy simulator or a real quantum computer. You've already seen on day 1 how to set/get a noisy backend. You'll see that noise messes things up pretty bad.

Running on real machine will introduce the problem of the qubit layout. You might want to change the `initial_layout` in the `execute_opts` so that your `varform` is not applying CNOT gates between qubits that are not connected. You know this needs to insert SWAP gates and this introduces more noise. Also covered in day 1.

To limit the effect of readout noise, you could add a `measure_filter` to your `evaluator`, so that each time you execute the `eval_circuits` you apply the filter to the results. Also covered in day 1.

Implement the simulatneous evaluation for bitwise commuting cliques or even for general commuting cliques.

Notebook by **Maxime Dion** <maxime.dion@usherbrooke.ca><br>
For the QSciTech-QuantumBC virtual workshop on gate-based quantum computing